# 유통 데이터 - 리텐션과 RFM 분석

## 데이터 살펴보기

In [1]:
import pandas as pd
import plotly.express as px

data = pd.read_csv('./data/ecommerce_data.csv')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 데이터 전처리

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [3]:
# 고객 분석을 할 것이므로 CustomerID가 없는 행은 제거
data.dropna(subset=['CustomerID'], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


In [4]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], format='%m/%d/%Y %H:%M')
data['CustomerID'] = data['CustomerID'].astype(int).astype(str)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  object        
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.9+ MB


In [5]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [6]:
data['date_ymd'] = data['InvoiceDate'].dt.date.astype('datetime64[ns]')
data['year'] = data['InvoiceDate'].dt.year
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date_ymd,year
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12-01,2010
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010-12-01,2010
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010


In [7]:
data['amount'] = data['Quantity'] * data['UnitPrice']
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date_ymd,year,amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12-01,2010,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010-12-01,2010,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34


In [8]:
data = data.query('Quantity > 0')
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  object        
 7   Country      397924 non-null  object        
 8   date_ymd     397924 non-null  datetime64[ns]
 9   year         397924 non-null  int32         
 10  amount       397924 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int32(1), int64(1), object(5)
memory usage: 34.9+ MB


## 분석

### 매출

In [9]:
amount_by_date = data.groupby('date_ymd')[['amount']].sum().reset_index()

fig = px.line(data_frame=amount_by_date, x='date_ymd', y='amount')
fig.show()

### 주문 고객수

In [10]:
customer_count_by_date = data.groupby('date_ymd')[['CustomerID']].nunique().reset_index().rename({'CustomerID':'customer_count'}, axis=1)

fig = px.line(data_frame=customer_count_by_date, x='date_ymd', y='customer_count')
fig.show()

### 주문 단가

In [11]:
amount_by_date.head()

,date_ymd,amount
0,2010-12-01,46376.49
1,2010-12-02,47316.53
2,2010-12-03,23921.71
3,2010-12-05,31771.60
4,2010-12-06,31215.64


In [12]:
invoice_count_by_date = data.groupby('date_ymd')[['InvoiceNo']].nunique().reset_index().rename({'InvoiceNo':'invoice_count'}, axis=1)
invoice_count_by_date.head()

,date_ymd,invoice_count
0,2010-12-01,121
1,2010-12-02,137
2,2010-12-03,57
3,2010-12-05,87
4,2010-12-06,94


In [13]:
invoice_amount = pd.merge(amount_by_date, invoice_count_by_date, on='date_ymd')
invoice_amount['amount_per_invoice'] = invoice_amount['amount'] / invoice_amount['invoice_count']
invoice_amount.head()

,date_ymd,amount,invoice_count,amount_per_invoice
0,2010-12-01,46376.49,121,383.276777
1,2010-12-02,47316.53,137,345.376131
2,2010-12-03,23921.71,57,419.679123
3,2010-12-05,31771.60,87,365.190805
4,2010-12-06,31215.64,94,332.081277


In [14]:
fig = px.line(data_frame=invoice_amount, x='date_ymd', y='amount_per_invoice')
fig.show()

## 리텐션 분석

In [15]:
retention_base = data[['CustomerID', 'InvoiceNo', 'date_ymd']].drop_duplicates()
retention_base['date_ym'] = retention_base['date_ymd'].dt.to_period('M')
retention_base.head()

,CustomerID,InvoiceNo,date_ymd,date_ym
0,17850,536365,2010-12-01,2010-12
7,17850,536366,2010-12-01,2010-12
9,13047,536367,2010-12-01,2010-12
21,13047,536368,2010-12-01,2010-12
25,13047,536369,2010-12-01,2010-12


In [16]:
print(min(data['date_ymd'].unique()))
print(max(data['date_ymd'].unique()))

2010-12-01 00:00:00
2011-12-09 00:00:00


In [17]:
# 12월 데이터를 포함하면 2011년 12월 데이터는 리텐션이 낮을 수 밖에 없으므로 12월 데이터 제외
retention_base = retention_base.query('date_ymd <= "2011-11-30"') 

In [18]:
date_ym_list = sorted(list(retention_base['date_ym'].unique()))
date_ym_list

[Period('2010-12', 'M'),
 Period('2011-01', 'M'),
 Period('2011-02', 'M'),
 Period('2011-03', 'M'),
 Period('2011-04', 'M'),
 Period('2011-05', 'M'),
 Period('2011-06', 'M'),
 Period('2011-07', 'M'),
 Period('2011-08', 'M'),
 Period('2011-09', 'M'),
 Period('2011-10', 'M'),
 Period('2011-11', 'M')]

In [19]:
from tqdm.notebook import tqdm

retention = pd.DataFrame()
for s in tqdm(date_ym_list):
    for t in date_ym_list:
        period_start = s
        period_target = t
        
        if period_start <= period_target:
            period_start_user = set(retention_base.query('date_ym == @period_start')['CustomerID'])
            period_target_user = set(retention_base.query('date_ym == @period_target')['CustomerID'])
            
            retained_users = period_start_user.intersection(period_target_user)
            retention_rate = len(retained_users) / len(period_start_user)
            
            temp = pd.DataFrame({'cohort':[period_start], 'date_ym':[period_target], 'retention_rate':[retention_rate]})
            
            retention = pd.concat([retention, temp])

  0%|          | 0/12 [00:00<?, ?it/s]

In [20]:
retention['cohort_size(month)'] = retention.apply(lambda x: (x['date_ym'] - x['cohort']).n, axis=1)
retention.head()

,cohort,date_ym,retention_rate,cohort_size(month)
0,2010-12,2010-12,1.000000,0
0,2010-12,2011-01,0.366102,1
0,2010-12,2011-02,0.323164,2
0,2010-12,2011-03,0.384181,3
0,2010-12,2011-04,0.362712,4


In [21]:
retention['cohort'] = retention['cohort'].astype(str)
retention['date_ym'] = retention['date_ym'].astype(str)

retention_final = pd.pivot_table(data=retention, index='cohort', columns='cohort_size(month)', values='retention_rate')
retention_final

cohort_size(month),0,1,2,3,4,5,6,7,8,9,10,11
cohort,,,,,,,,,,,,
2010-12,1.0,0.366102,0.323164,0.384181,0.362712,0.397740,0.362712,0.349153,0.353672,0.395480,0.374011,0.502825
2011-01,1.0,0.353576,0.430499,0.383266,0.457490,0.419703,0.391363,0.384615,0.438596,0.417004,0.514170,NaN
2011-02,1.0,0.382586,0.375989,0.468338,0.428760,0.405013,0.402375,0.440633,0.414248,0.485488,NaN,NaN
2011-03,1.0,0.312115,0.441478,0.401437,0.377823,0.360370,0.421971,0.390144,0.474333,NaN,NaN,NaN
2011-04,1.0,0.429907,0.419393,0.407710,0.387850,0.450935,0.417056,0.491822,NaN,NaN,NaN,NaN
2011-05,1.0,0.388258,0.407197,0.405303,0.442235,0.416667,0.501894,NaN,NaN,NaN,NaN,NaN
2011-06,1.0,0.368315,0.409687,0.467205,0.435923,0.517659,NaN,NaN,NaN,NaN,NaN,NaN
2011-07,1.0,0.408851,0.457323,0.457323,0.521602,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08,1.0,0.454545,0.449198,0.534759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
fig = px.imshow(retention_final, text_auto='.2%', color_continuous_scale='Burg')
fig.show()

In [23]:
retention_curve = retention.groupby('cohort_size(month)')[['retention_rate']].mean().reset_index()
retention_curve

,cohort_size(month),retention_rate
0,0,1.000000
1,1,0.391502
2,2,0.423605
3,3,0.434391
4,4,0.426799
5,5,0.424012
6,6,0.416228
7,7,0.411273
8,8,0.420212
9,9,0.432657


In [24]:
fig = px.line(data_frame=retention_curve, x='cohort_size(month)', y='retention_rate', title='retention curve')
fig.update_yaxes(tickformat='.2%')
fig.show()

## RFM analysis

In [25]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date_ymd,year,amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12-01,2010,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010-12-01,2010,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34


In [26]:
today_date = max(data['date_ymd'])

rfm = data.groupby('CustomerID').agg({'InvoiceDate': lambda x: (today_date - x.max()).days,     # 오늘로부터 며칠이 지났는지
                                      'amount': lambda x: x.sum()       # 주문금액
                                      })
rfm.columns = ['recency', 'monetary']
rfm.head()

,recency,monetary
CustomerID,,
12346,324,77183.60
12347,1,4310.00
12348,74,1797.24
12349,17,1757.55
12350,309,334.40


In [27]:
pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])

CustomerID
12346    1
12347    5
12348    2
12349    4
12350    1
        ..
18280    1
18281    1
18282    5
18283    5
18287    3
Name: recency, Length: 4339, dtype: category
Categories (5, int64): [5 < 4 < 3 < 2 < 1]

In [28]:
rfm['recency_score'] = pd.qcut(rfm['recency'], 3, labels=[3, 2, 1])
rfm['monetary_score'] = pd.qcut(rfm['monetary'], 3, labels=[1, 2, 3])
rfm['rm_score'] = rfm['recency_score'].astype(str) + rfm['monetary_score'].astype(str)
rfm.reset_index(inplace=True)
rfm

,CustomerID,recency,monetary,recency_score,monetary_score,rm_score
0,12346,324,77183.60,1,3,13
1,12347,1,4310.00,3,3,33
2,12348,74,1797.24,2,3,23
3,12349,17,1757.55,3,3,33
4,12350,309,334.40,1,1,11
...,...,...,...,...,...,...
4334,18280,276,180.60,1,1,11
4335,18281,179,80.82,1,1,11
4336,18282,6,178.05,3,1,31
4337,18283,2,2094.88,3,3,33


In [29]:
rm_score = rfm.groupby('rm_score')[['CustomerID']].nunique().reset_index().rename({'CustomerID':'customer_count'}, axis=1)
rm_score

,rm_score,customer_count
0,11,790
1,12,481
2,13,170
3,21,407
4,22,546
5,23,457
6,31,250
7,32,419
8,33,819


In [30]:
def categorize_customer(score):
    if score == '33':
        return '최우수'     # 최신성, 구매 모두 매우 높음
    elif score in ['32', '23', '22']:
        return '우수'       # 최신성, 구매 모두 높음
    elif score == '11':
        return '휴면'       # 최신성, 구매 모두 낮음
    elif score in ['12', '13']:
        return '이탈 방지'   # 구매는 높으나 최신성은 낮음 -> 다시 불러 들어야 함
    elif score in ['31', '21']:
        return '구매 유도'   # 최신성은 높으나 구매는 낮음 -> 구매를 유도해야 함
    
rm_score['category'] = rm_score['rm_score'].apply(categorize_customer) 

In [31]:
fig = px.treemap(data_frame=rm_score, path=['category'], values='customer_count', color_discrete_sequence=px.colors.qualitative.Pastel1)
fig.show()